In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [4]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls=["https://brainlox.com/courses/category/technical"]

In [5]:
loader = UnstructuredURLLoader(urls=urls)
data=loader.load()

data[0]


Document(metadata={'source': 'https://brainlox.com/courses/category/technical'}, page_content='Courses\n\nTechnical\n\nAcademic\n\nLanguage\n\nMusic\n\nLifestyle\n\nBook a Free Demo Now\n\nSign In\n\nFAQ\n\nContact Us\n\nHome\n\nCourses\n\nCourses\n\nWe found great courses available for you\n\n$20 per session\n\nLEARN SCRATCH PROGRAMING\n\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want\n\n16 Lessons\n\nView Details\n\n$30 per session\n\nLEARN CLOUD COMPUTING BASICS-AWS\n\nIn this course we are going to cover the basics and the most important services on AWS, At the end\n\n20 Lessons\n\nView Details\n\n$20 per session\n\nLEARN MOBILE DEVELOPMENT\n\nMobile application development is the process of creating software applications that run on a mobil\n\n24 Lessons\n\nView Details\n\n$20 per session\n\nLEARN CORE JAVA PROGRAMMING ONLINE\n\nJava is a very popular high-level, class-based, object-oriented programming language that is design\n

In [6]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(data,embeddings)

In [7]:
vectorstoredb

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

In [9]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [16]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [19]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(data)
vectorstoredb=FAISS.from_documents(data,embeddings)
retriever=vectorstoredb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x308f76360>, search_kwargs={})

In [21]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"How many lessons are there in LEARN CORE JAVA PROGRAMMING ONLINE"})
response

{'input': 'How many lessons are there in LEARN CORE JAVA PROGRAMMING ONLINE',
 'context': [Document(metadata={'source': 'https://brainlox.com/courses/category/technical'}, page_content='Courses\n\nTechnical\n\nAcademic\n\nLanguage\n\nMusic\n\nLifestyle\n\nBook a Free Demo Now\n\nSign In\n\nFAQ\n\nContact Us\n\nHome\n\nCourses\n\nCourses\n\nWe found great courses available for you\n\n$20 per session\n\nLEARN SCRATCH PROGRAMING\n\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want\n\n16 Lessons\n\nView Details\n\n$30 per session\n\nLEARN CLOUD COMPUTING BASICS-AWS\n\nIn this course we are going to cover the basics and the most important services on AWS, At the end\n\n20 Lessons\n\nView Details\n\n$20 per session\n\nLEARN MOBILE DEVELOPMENT\n\nMobile application development is the process of creating software applications that run on a mobil\n\n24 Lessons\n\nView Details\n\n$20 per session\n\nLEARN CORE JAVA PROGRAMMING ONLINE\n\nJava is a 